In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
random_state = 7
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [13]:
import pickle
with open("genes_to_keep.pickle", 'rb') as f:
    keep_pickle = pickle.load(f)

keep = []
for item in keep_pickle:
    keep.append(item)



keep.append("target")

In [14]:
len(keep)

12

In [15]:
tr = "trainset_ALLvsHealhy_80_target_in"
ts = "testset_ALLvsHealthy_20_target_in" 

# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

path = os.getcwd()
parent = os.path.dirname(path)

df_whole_train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
df_whole_test = pd.read_csv(parent + '\Data\\' + ts + ".csv")

In [16]:
df_reduced_train = df_whole_train[keep]
df_reduced_test = df_whole_test[keep]

In [17]:
df_reduced_train.to_csv(parent + '\Data\\' + "trainset_ALL_vs_Healthy_reduced_25_80_target_in.csv", index=False)
df_reduced_test.to_csv(parent + '\Data\\' + "testset_ALL_vs_Healthy_reduced_25_20_target_in.csv", index=False)